In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "enrich_cancel_not_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
# Note: this key is case sensitive!!
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

transform = Transform()

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Cancelled Not Before Active enrichment. Assigns hierarchy values in cases where cancelled status is NOT reported before first active shipment. (either cancel after active, or no active reported).  This is used as part of the Fill Rate enrichment. See logic diagram below:

<img src = 'assets/cancel_not_before_active.svg' style="width:800px;">

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[4]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

In [ ]:
df.shape

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

### DATA CLEANING: ADDRESS THIS SECTION BEFORE PIPELINE INTEGRATION

In [ ]:
if input_pharma == 'sun':
    trans_id = 'pharm_transaction_id'
    brand = 'medication'
    patient_id = 'msa_patient_id'
    pharmacy = 'pharm_code'
    status_date = 'status_date'
    referral_date = 'ref_date'
    status =  'status_code'
    substatus =  'sub_status'
    datetime = '%Y%m%d'
    pending_status_code = 'PENDING'
    active_status_code = 'ACTIVE'
    active_substatus_code = 'SHIPMENT'
    cancel_status_code = 'CANCELLED'
    payer_substatus_code = 'INSURANCE DENIED'
    transfer_substatus_code = 'TRANSFER SP'

elif input_pharma == 'bi':
    trans_id = 'pmcTransactionId'
    brand = 'medication'
    patient_id = 'pmcPatientID'
    pharmacy = 'pharmName'
    status_date = 'statusDate'
    referral_date = 'refDate'
    status =  'statusCode'
    substatus =  'custStatusCode'
    datetime = '%Y-%b-%d'
    pending_status_code = 'PENDING'
    active_status_code = 'ACTIVE'
    active_substatus_code = 'S01'
    cancel_status_code = 'CANCELLED'
    active_hierarchy = 'ACTIVE - S01'
    payer_substatus_code = 'INSURANCE DENIED'
    transfer_substatus_code = 'TRANSFER SP'

In [ ]:
# Create variables for hierarchy assignments

payer_hierarchy = 'Payer'
transfer_greater2_hierarchy = 'Transferred > 2 Days'
transfer_less2_hierarchy = 'Transferred <= 2 Days'
pending_greater7_hierarchy = 'Pending >= 7 Days'
pending_4to6_hierarchy = 'Pending 4 to 6 Days'
pending_less3_hierarchy = 'Pending <= 3 Days'

In [ ]:
# CLEAN DATA - This step should not be necessary once transform is integrated into pipeline.
#    Extract and map relevant columns
#    Convert dates to datetime format
#    Extract brand from medication
#    Convert substatuses to uppercase
#    Populate null referral dates with the min(status_date) for that patient/pharmacy/brand.
    
def clean_data(cust_input_df, datetime, trans_id, brand, patient_id, pharmacy, status_date, referral_date, status, substatus):

    clean_df = (
        cust_input_df
        .loc[:,
             [trans_id,
              patient_id,
              pharmacy,
              brand,
              status_date,
              referral_date,
              status,
              substatus]
            ]
        .rename(
            columns={
                trans_id:'trans_id',
                patient_id:'patient_id',
                pharmacy:'pharmacy',
                brand:'brand',
                status_date:'status_date',
                referral_date:'referral_date',
                status:'status_code',
                substatus:'substatus_code'}
        )
        .assign(status_date = lambda x: (
            pd.to_datetime(
                x.status_date.str[:8].astype(str),
                format=datetime,
                errors='coerce'
            )
        ))
        .assign(min_status_date = lambda x: (
            x.groupby(['patient_id','pharmacy','brand'])['status_date']
            .transform(min)
        ))
        .fillna(value={'referral_date':'min_status_date'})
        .assign(referral_date = lambda x: (
            pd.to_datetime(
                x.referral_date.str[:8].astype(str),
                format=datetime,
                errors='coerce'
            )
        ))
        .dropna()
        .assign(brand = lambda x: (
            x['brand'].apply(lambda x: x.split()[0].strip())
        ))
        .assign(status_code = lambda x: (
            x.status_code.str.upper()
        ))
        .assign(substatus_code = lambda x: (
            x.substatus_code.str.upper()
        ))

        .drop(['min_status_date'],axis=1)
        .drop_duplicates()
        .sort_values(
            by=['patient_id', 'pharmacy', 'brand', 'status_date','status_code'],
            ascending=[True, True, True, True, False])
        .reset_index(drop=True)
    )

    return clean_df

In [ ]:
clean_df = clean_data(
    df,
    datetime,
    trans_id,
    brand,
    patient_id,
    pharmacy,
    status_date,
    referral_date,
    status,
    substatus
)

clean_df.head()

In [ ]:
clean_df.shape

### APPLY TRANSFORM LOGIC

In [ ]:
# Assign Patient Journey (pj_id), Patient Journey Step (pj_step) and Patient Journey Phase (pj_phase) identifiers
# (These IDs are used for calculation purposes only.  They will not be published)

def pj(clean_df):
    pj_df = (
        clean_df
        .assign(pj_id = lambda x: (
            x.groupby(['patient_id','pharmacy','brand']).grouper.group_info[0]
        ))
        .assign(pj_step = lambda x: x.index)
        .assign(pj_phase = lambda x:(
            np.where((x['pj_id'] == x['pj_id'].shift(1))
                     & (x['status_code'] == x['status_code'].shift(1)),
                     0,
                     1
                    )
            .cumsum()
        ))
    )
    return pj_df

In [ ]:
# Filter to only include patient journeys where at least 1 cancelled status is reported

def cancel(pj_df):
    cancel_df = (
        pj_df
        .loc[lambda x: (
            x['pj_id'].isin(x
                            .loc[x['status_code'] == cancel_status_code]
                            .pj_id
                            .drop_duplicates()
                            .tolist()
                           )
        )]
    )
    return cancel_df

In [ ]:
# Filter to only include patient journeys where at least 1 of the following is true:
#    a) No active shipment reported OR
#    b) First active shipment occurs PRIOR to cancelled status

def cancel_not_before_active(cancel_df):
    cancel_not_before_active_df = (
        cancel_df
        .assign(active_phase = lambda x: (
            x.loc[x['substatus_code'] == active_substatus_code].groupby(['pj_id'])['pj_phase']
            .transform(min)
        ))
        .assign(active_phase = lambda x: (
            x.groupby(['pj_id'], sort=False)['active_phase']
            .transform(lambda x: x.ffill())
        ))
        .assign(active_phase = lambda x: (
            x.groupby(['pj_id'], sort=False)['active_phase']
            .transform(lambda x: x.bfill())
        ))
        .assign(cancel_flag = lambda x: (
            np.where(
                (x['status_code'] == cancel_status_code)
                & (
                    (x['pj_phase'] > x['active_phase'])
                    | (x['active_phase'].isnull())
                ),
                1,
                0
            )
        ))
        .loc[lambda x: (
            x['pj_id'].isin(x
                            .loc[x['cancel_flag'] == 1]
                            .pj_id
                            .drop_duplicates()
                            .tolist()
                           )
        )]
        .drop(['active_phase'], axis=1)
    )
    return cancel_not_before_active_df

In [ ]:
# Find all pending phases and time spent in each pending phase
# Forward fill so that all subsequent cancels adopt a "prior_pending_time".

def prior_pending(cancel_not_before_active_df):
    prior_pending_df = (
        cancel_not_before_active_df
        .assign(first_journey_step = lambda x: (
            x.groupby(['pj_id'])['pj_step']
            .transform(min)
        ))
        .assign(first_phase_step = lambda x: (
            x.groupby(['pj_phase'])['pj_step']
            .transform(min)
        ))
        .assign(prev_phase = lambda x: (
            np.where(
                (x['pj_step'] != x['first_journey_step']) & (x['pj_step'] == x['first_phase_step']),
                (
                    x.groupby(['pj_phase'])['status_code']
                    .transform(min).shift(1)
                ),
                None)
        ))
        .assign(prior_pending_time = lambda x: (
            (x['status_date'] - 
            pd.to_datetime(
                np.where(
                    x['prev_phase'] == pending_status_code,
                    (
                        x.groupby(['pj_id','pj_phase'])['status_date']
                        .transform(min).shift(1)
                    ),
                    pd.NaT)
            )) / np.timedelta64(1,'D')
        ))
        .assign(prior_pending_time = lambda x: (
            x.groupby(['pj_id'], sort=False)['prior_pending_time']
            .transform(lambda x: x.ffill())
            ))
        .drop(['first_journey_step','first_phase_step','prev_phase'], axis=1)
    )
    return prior_pending_df

In [ ]:
## Find the min(referral_date) for each patient journey (This is used in the hierarchy logic for Transfers)

def ref(prior_pending_df):
    ref_df = (
        prior_pending_df
        .assign(min_ref_date = lambda x: (
            x.groupby(['pj_id'])['referral_date']
            .transform(min)
        ))
        .assign(ref_time = lambda x: (
            (x['status_date'] - x['min_ref_date']) / np.timedelta64(1, 'D')
        ))
        .drop(['min_ref_date'], axis=1)
    )
    return ref_df

In [ ]:
# Apply logic to determine patient journey hierarchy. See logic diagram in transform description.

def hierarchy(ref_df):
    hierarchy_df = (
        ref_df
        .assign(hierarchy = lambda x:(
            np.where(
                x['cancel_flag'] == 0,
                None,
                np.where(
                    x['substatus_code'] == payer_substatus_code,
                    payer_hierarchy,
                    np.where(
                        x['substatus_code'] == transfer_substatus_code,
                        np.where(
                            x['ref_time'] > 2,
                            transfer_greater2_hierarchy,
                            transfer_less2_hierarchy
                        ),
                        np.where(
                            x['prior_pending_time'] >= 7,
                            pending_greater7_hierarchy,
                            np.where(
                                x['prior_pending_time'] > 3,
                                pending_4to6_hierarchy,
                                pending_less3_hierarchy
                            )
                        )
                    )
                )
            )
        ))
    )
    return hierarchy_df

In [ ]:
pj_df = pj(clean_df)

pj_df.head()

In [ ]:
hierarchy_df = (
    pj_df
    .pipe(cancel)
    .pipe(cancel_not_before_active)
    .pipe(prior_pending)
    .pipe(ref)
    .pipe(hierarchy)
)

hierarchy_df.head()

In [ ]:
final_dataframe = (
    pd.merge(
        pj_df,
        hierarchy_df.loc[
            hierarchy_df['cancel_flag'] == 1,
            ['pj_id','pj_step','hierarchy']],
        how='left',
        on=['pj_id', 'pj_step']
    )
    .drop(['pj_id', 'pj_step', 'pj_phase'], axis=1)
)

final_dataframe.head()

In [ ]:
final_dataframe.shape

### TEST TRANSFORM OUTPUT

In [ ]:
# TEST 1: Create a "test" dataframe with expected results

test_data = ([
    [0, 0, active_status_code, 'READY', None],
    [0, 1, cancel_status_code, transfer_substatus_code, transfer_less2_hierarchy],
    [0, 5, cancel_status_code, 'OTHER', pending_less3_hierarchy],
    [0, 5, cancel_status_code, payer_substatus_code, payer_hierarchy],
    [0, 6, pending_status_code, 'OTHER', None],
    [0, 10, cancel_status_code, 'OTHER', pending_4to6_hierarchy],
    [0, 12, pending_status_code, 'OTHER', None],
    [0, 12, cancel_status_code, 'OTHER', pending_less3_hierarchy],
    [0, 13, cancel_status_code, payer_substatus_code, payer_hierarchy],
    [14, 14, cancel_status_code, 'OTHER', pending_less3_hierarchy],
    [14, 15, cancel_status_code, transfer_substatus_code, transfer_greater2_hierarchy]
])

test_df = (
    pd.DataFrame(test_data, columns = ['referral_date','status_date','status_code','substatus_code', 'expected_hierarchy'])
    .assign(patient_id = 123)
    .assign(pharmacy = 'ABC')
    .assign(brand = 'A')
    .assign(status_date = lambda x: (
        pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x.status_date, unit='d')
    ))
    .assign(referral_date = lambda x: (
        pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x.referral_date, unit='d')
    ))
    
)

test_df

In [ ]:
# Apply transform to test dataframe

pj_test = pj(test_df)

test_output = (
    pj_test
    .pipe(cancel)
    .pipe(cancel_not_before_active)
    .pipe(prior_pending)
    .pipe(ref)
    .pipe(hierarchy)
)

final_dataframe_test = (
    pd.merge(
        pj_test,
        test_output.loc[
            test_output['cancel_flag'] == 1,
            ['pj_id','pj_step','hierarchy']],
        how='left',
        on=['pj_id', 'pj_step']
    )
    .drop(['pj_id', 'pj_step', 'pj_phase'], axis=1)
)

final_dataframe_test

In [ ]:
# Check that results match expectations

test1 = (
    final_dataframe_test
    .assign(passfail = lambda x: np.where(
        (x.hierarchy == x.expected_hierarchy) | (x.hierarchy.isnull() & x.expected_hierarchy.isnull()),
        True,
        False
    ))
    .passfail
    .all()
)

test1

In [ ]:
# TEST 2: Check that final dataframe has the same number of rows as the input dataframe

test2 = (final_dataframe.shape[0] == clean_df.shape[0])

test2

In [ ]:
# TEST 3: Check that all cancel statuses NOT before an active shipment have 1 of the 6 cancel_not_before_active hierarchies assigned to them.

hierarchy_list = [payer_hierarchy, transfer_greater2_hierarchy, transfer_less2_hierarchy, pending_greater7_hierarchy, pending_4to6_hierarchy, pending_less3_hierarchy]

first_active_status = (
    final_dataframe
    .assign(pj_id = lambda x: x.groupby(['patient_id','pharmacy','brand']).grouper.group_info[0])
    .assign(pj_step = lambda x: x.index)
    .assign(active_status_date = lambda x: (
        pd.to_datetime(np.where(
            x['substatus_code']==active_substatus_code,
            x['status_date'],
            pd.NaT
    ))))
    .assign(first_active_status_date = lambda x: (
        x.groupby(['pj_id'])['active_status_date']
        .transform(min)
    ))
    .drop(['active_status_date'], axis=1)
)

test3 = (first_active_status   
    .loc[lambda x: (
        ((x['status_date'] > x['first_active_status_date'])
         |
         (x['first_active_status_date'].isnull())
        )
        &
        (x['status_code'] == cancel_status_code)
        &
        (~x['hierarchy'].isin(hierarchy_list))
    )]
)

test3 = (test3.shape[0] == 0)

test3

In [ ]:
# TEST 4: Check that all statuses BEFORE an active shipment do NOT any of the 6 cancel_not_before_active hierarchies assigned to them.

test4 = (
    first_active_status   
    .loc[lambda x: (
        (x['status_date'] < x['first_active_status_date'])
        &
        (x['hierarchy'].isin(hierarchy_list))
    )]
)

test4 = (test4.shape[0] == 0)

test4

In [ ]:
# FINAL TEST: Did all 4 tests pass?

test1 & test2 & test3 & test4

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()